# Train Model

**INPUT**: "./data/1finalDataset.csv"

**OUTPUT**: Outputs the XGBoostModels "./models/best_xgb_model.json"

In this notebook, we take the final dataset (which contains all the tennis statistics), and we train several models with it (Random Forest, XGBoost, Neural Net). Then, we will save the best models to the models folder.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import plot_tree
from sklearn import tree
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)

In [ ]:
import random
import numpy as np
import torch
def seed_everything(seed: int = 41):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # si multi-GPU

    # Pour forcer la reproductibilité sur CUDA (moins perf mais stable)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Exemple :
seed_everything(41)


In [2]:
final_dataset = pd.read_csv("./data/1finalDataset.csv")
final_dataset.columns

Index(['AGE_DIFF', 'ATP_POINTS_DIFF', 'ATP_RANK_DIFF', 'BEST_OF', 'DRAW_SIZE',
       'ELO_DIFF', 'ELO_GRAD_LAST_100_DIFF', 'ELO_GRAD_LAST_10_DIFF',
       'ELO_GRAD_LAST_200_DIFF', 'ELO_GRAD_LAST_25_DIFF',
       'ELO_GRAD_LAST_3_DIFF', 'ELO_GRAD_LAST_50_DIFF', 'ELO_GRAD_LAST_5_DIFF',
       'ELO_SURFACE_DIFF', 'H2H_DIFF', 'H2H_SURFACE_DIFF', 'HEIGHT_DIFF',
       'N_GAMES_DIFF', 'P_1ST_IN_LAST_100_DIFF', 'P_1ST_IN_LAST_10_DIFF',
       'P_1ST_IN_LAST_200_DIFF', 'P_1ST_IN_LAST_25_DIFF',
       'P_1ST_IN_LAST_3_DIFF', 'P_1ST_IN_LAST_50_DIFF', 'P_1ST_IN_LAST_5_DIFF',
       'P_1ST_WON_LAST_100_DIFF', 'P_1ST_WON_LAST_10_DIFF',
       'P_1ST_WON_LAST_200_DIFF', 'P_1ST_WON_LAST_25_DIFF',
       'P_1ST_WON_LAST_3_DIFF', 'P_1ST_WON_LAST_50_DIFF',
       'P_1ST_WON_LAST_5_DIFF', 'P_2ND_WON_LAST_100_DIFF',
       'P_2ND_WON_LAST_10_DIFF', 'P_2ND_WON_LAST_200_DIFF',
       'P_2ND_WON_LAST_25_DIFF', 'P_2ND_WON_LAST_3_DIFF',
       'P_2ND_WON_LAST_50_DIFF', 'P_2ND_WON_LAST_5_DIFF',
       'P_ACE_

In [3]:
import pandas as pd

# Récupère toutes les valeurs uniques
all_ids = pd.unique(final_dataset[["p1_id", "p2_id"]].values.ravel())

# Crée un mapping {id_original -> id_compact}
id_map = {old_id: new_id for new_id, old_id in enumerate(all_ids)}

# Applique le mapping
final_dataset["p1_id"] = final_dataset["p1_id"].map(id_map)
final_dataset["p2_id"] = final_dataset["p2_id"].map(id_map)

# Nombre de noeuds réels
num_nodes = len(all_ids)
print("Nombre de joueurs:", num_nodes)


Nombre de joueurs: 1932


In [4]:
date = 20240101
train_df = final_dataset[final_dataset["date"] < date].copy()
test_df  = final_dataset[final_dataset["date"] >= date].copy()
print(len(train_df))
print(len(test_df))

92429
2946


In [5]:
features_cols = [
    'AGE_DIFF', 'ATP_POINTS_DIFF', 'ATP_RANK_DIFF', 'BEST_OF', 'DRAW_SIZE',
       'ELO_DIFF', 'ELO_GRAD_LAST_100_DIFF', 'ELO_GRAD_LAST_10_DIFF',
       'ELO_GRAD_LAST_200_DIFF', 'ELO_GRAD_LAST_25_DIFF',
       'ELO_GRAD_LAST_3_DIFF', 'ELO_GRAD_LAST_50_DIFF', 'ELO_GRAD_LAST_5_DIFF',
       'ELO_SURFACE_DIFF', 'H2H_DIFF', 'H2H_SURFACE_DIFF', 'HEIGHT_DIFF',
       'N_GAMES_DIFF', 'P_1ST_IN_LAST_100_DIFF', 'P_1ST_IN_LAST_10_DIFF',
       'P_1ST_IN_LAST_200_DIFF', 'P_1ST_IN_LAST_25_DIFF',
       'P_1ST_IN_LAST_3_DIFF', 'P_1ST_IN_LAST_50_DIFF', 'P_1ST_IN_LAST_5_DIFF',
       'P_1ST_WON_LAST_100_DIFF', 'P_1ST_WON_LAST_10_DIFF',
       'P_1ST_WON_LAST_200_DIFF', 'P_1ST_WON_LAST_25_DIFF',
       'P_1ST_WON_LAST_3_DIFF', 'P_1ST_WON_LAST_50_DIFF',
       'P_1ST_WON_LAST_5_DIFF', 'P_2ND_WON_LAST_100_DIFF',
       'P_2ND_WON_LAST_10_DIFF', 'P_2ND_WON_LAST_200_DIFF',
       'P_2ND_WON_LAST_25_DIFF', 'P_2ND_WON_LAST_3_DIFF',
       'P_2ND_WON_LAST_50_DIFF', 'P_2ND_WON_LAST_5_DIFF',
       'P_ACE_LAST_100_DIFF', 'P_ACE_LAST_10_DIFF', 'P_ACE_LAST_200_DIFF',
       'P_ACE_LAST_25_DIFF', 'P_ACE_LAST_3_DIFF', 'P_ACE_LAST_50_DIFF',
       'P_ACE_LAST_5_DIFF', 'P_BP_SAVED_LAST_100_DIFF',
       'P_BP_SAVED_LAST_10_DIFF', 'P_BP_SAVED_LAST_200_DIFF',
       'P_BP_SAVED_LAST_25_DIFF', 'P_BP_SAVED_LAST_3_DIFF',
       'P_BP_SAVED_LAST_50_DIFF', 'P_BP_SAVED_LAST_5_DIFF',
       'P_DF_LAST_100_DIFF', 'P_DF_LAST_10_DIFF', 'P_DF_LAST_200_DIFF',
       'P_DF_LAST_25_DIFF', 'P_DF_LAST_3_DIFF', 'P_DF_LAST_50_DIFF',
       'P_DF_LAST_5_DIFF', 'WIN_LAST_100_DIFF',
       'WIN_LAST_10_DIFF', 'WIN_LAST_200_DIFF', 'WIN_LAST_25_DIFF',
       'WIN_LAST_3_DIFF', 'WIN_LAST_50_DIFF', 'WIN_LAST_5_DIFF'
]

## Split Training vs Testing Data

We'll shuffle the data, and do a 85% split between training and testing data.

In [6]:
import torch
from torch_geometric.data import TemporalData
from torch_geometric.loader import TemporalDataLoader
def build_temporal_data(df_subset):
    return TemporalData(
        src = torch.tensor(df_subset["p1_id"].values, dtype=torch.long),
        dst = torch.tensor(df_subset["p2_id"].values, dtype=torch.long),
        t   = torch.tensor(df_subset["t_days"].values, dtype=torch.long),
        msg = torch.tensor(df_subset[features_cols].values, dtype=torch.float),
        y   = torch.tensor(df_subset["RESULT"].values, dtype=torch.float),
        closeness = torch.tensor(df_subset["score_closeness"].values,dtype=torch.float)
    )

We need to map the result column to string values (since that's what the sklearn library requires I'm pretty sure)

In [7]:
full_data = build_temporal_data(final_dataset)
train_data = build_temporal_data(train_df)
test_data  = build_temporal_data(test_df)

for data in (full_data,train_data, test_data):
    data.src = data.src.long()    # src en ints longs
    data.dst = data.dst.long()    # dst en ints longs
    data.t   = data.t.long()     # timestamps en floats
    data.msg = data.msg.float()   # features en floats
    data.y   = data.y.float()     # labels en floats
    data.closeness = data.closeness.float()
# 4) Déplacer t et msg sur GPU
device = "cuda"
for data in (full_data,train_data, test_data):
    data.t   = data.t.to(device)
    data.msg = data.msg.to(device)
# 6. DataLoaders pour entraînement
train_loader = TemporalDataLoader(train_data, batch_size=32, neg_sampling_ratio=0)
test_loader  = TemporalDataLoader(test_data, batch_size=32, neg_sampling_ratio=0)


In [8]:

for batch in train_loader:
    print("=== Nouveau batch ===")
    print("src:", batch.src)
    print("dst:", batch.dst)
    print("t:", batch.t)
    print("msg:", batch.msg)
    print("y:", batch.y)
    print("Closeness:", batch.closeness)
    break

=== Nouveau batch ===
src: tensor([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30,  2,  6,
        10, 12, 19, 22, 24, 29,  0, 27,  5, 22,  5, 32, 34, 36])
dst: tensor([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31,  0,  5,
         8, 15, 16, 21, 27, 30,  5, 29, 15, 29, 22, 33, 35, 37])
t: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
msg: tensor([[ 0.9601,  0.5384, -0.5188,  ..., -0.0022,  0.0043, -0.0023],
        [-1.9789,  0.1350, -0.9501,  ..., -0.0022,  0.0043, -0.0023],
        [-0.7111, -0.1837,  1.0249,  ..., -0.0022,  0.0043, -0.0023],
        ...,
        [ 0.6143,  1.0605, -0.3902,  ..., -0.0022,  0.0043, -0.0023],
        [-0.5575,  0.1548, -0.1783,  ..., -0.0022,  0.0043, -0.0023],
        [-1.0377, -0.1670,  0.3741,  ..., -0.0022,  0.0043, -0.0023]],
       device='cuda:0')
y: tensor([1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0.,

In [9]:
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import json

class History:
    def __init__(self, outdir="runs/curves", hparams=None):
        """
        hparams: dict optionnel avec les hyperparamètres 
                 (ex: {"lr":4e-4,"weight_decay":1e-5,"memory_dim":128,...})
        """
        self.outdir = Path(outdir)
        self.outdir.mkdir(parents=True, exist_ok=True)
        self.rows = []
        self.prec_at_rows = []
        self.detail_rows = []
        self.hparams = hparams if hparams is not None else {}

    def log_epoch(self, epoch,
                  train_loss, train_ap, train_prec,
                  val_loss,   val_ap,   val_prec,
                  prec_at=None):
        self.rows.append({
            "epoch": epoch,
            "train_loss": float(train_loss),
            "train_ap": float(train_ap),
            "train_prec@0.5": float(train_prec),
            "val_loss": float(val_loss),
            "val_ap": float(val_ap),
            "val_prec@0.5": float(val_prec),
        })
        if prec_at is not None:
            self.prec_at_rows.append(
                {"epoch": epoch, **{f"@{k}": float(v) for k, v in prec_at.items()}}
            )

    def save_tables(self):
        pd.DataFrame(self.rows).to_csv(self.outdir / "metrics_history.csv", index=False)
        if self.prec_at_rows:
            pd.DataFrame(self.prec_at_rows).to_csv(self.outdir / "precision_at_history.csv", index=False)
        if self.detail_rows:
            pd.DataFrame(self.detail_rows).to_csv(self.outdir / "predicted_dates.csv", index=False)
        # 💾 Sauvegarde aussi les hyperparamètres dans un JSON
        if self.hparams:
            with open(self.outdir / "hparams.json","w") as f:
                json.dump(self.hparams,f,indent=2)

    def _plot_and_save(self, x, y, ylabel, fname):
        plt.figure()
        plt.plot(x, y)
        plt.xlabel("epoch")
        plt.ylabel(ylabel)
        plt.grid(True, linestyle="--", linewidth=0.5)
        plt.tight_layout()
        plt.savefig(self.outdir / fname, dpi=200)
        plt.close()

    def save_plots(self):
        df = pd.DataFrame(self.rows)
        x = df["epoch"].values
        self._plot_and_save(x, df["train_loss"].values, "train_loss", "curve_train_loss.png")
        self._plot_and_save(x, df["val_loss"].values,   "val_loss",   "curve_val_loss.png")
        self._plot_and_save(x, df["train_ap"].values,   "train_AP",   "curve_train_ap.png")
        self._plot_and_save(x, df["val_ap"].values,     "val_AP",     "curve_val_ap.png")
        self._plot_and_save(x, df["train_prec@0.5"].values, "train_precision@0.5", "curve_train_prec.png")
        self._plot_and_save(x, df["val_prec@0.5"].values,   "val_precision@0.5",   "curve_val_prec.png")

    def save_all(self):
        self.save_tables()
        self.save_plots()


In [ ]:
import matplotlib.pyplot as plt
from itertools import product

from torch_geometric.nn import TGNMemory
from torch_geometric.nn.models.tgn import (
    LastAggregator,
    LastNeighborLoader,
    IdentityMessage
)
from tgn.model import MultiLayerTimeAwareGNN,MessageMLP,WinPredictorMLP,WinPredictor,SmallWinPredictor
from tgn.utils import train,evaluate,compute_alpha,train_debug,train_debug2
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Paramètres
memory_dim = 128
time_dim   = 32
embedding_dim = 128
in_channels = 128
hidden_channels = 32

num_layers = 2
heads = 4
dropout= 0.4
learning_rates = [1e-3, 4e-4]
weight_decays = [1e-4, 5e-4]
hidden_variants = [[512, 64], [256, 64]]

# Générer toutes les combinaisons
grid = list(product(learning_rates, weight_decays, hidden_variants))
run_id = 0
for lr, wd, hidden in grid:
    run_id += 1
    print(f"\n=== RUN {run_id} | lr={lr} | wd={wd} | hidden={hidden} ===")

    msg_dim = full_data.msg.size(-1)

    memory = TGNMemory(
        num_nodes=num_nodes,
        raw_msg_dim=msg_dim,
        memory_dim=memory_dim,
        time_dim=time_dim,
        message_module=MessageMLP(msg_dim, memory_dim, time_dim,2*memory_dim),
        aggregator_module=LastAggregator(),
    ).to(device)

    gnn = MultiLayerTimeAwareGNN(in_channels,memory_dim,hidden_channels, 
                                 embedding_dim, msg_dim, memory.time_enc,
                                 num_layers,heads,dropout).to(device)
    
    win_pred = SmallWinPredictor(
        embed_dim=embedding_dim,
        match_dim=msg_dim,
        hidden = hidden 
    ).to(device)

    total_params = 0
    for model in [memory, gnn, win_pred]:
        model_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"{model.__class__.__name__} params: {model_params:,}")
        total_params += model_params

    print(f"Total parameters: {total_params:,}")



    optimizer = torch.optim.AdamW(
        list(memory.parameters()) + list(gnn.parameters()) + list(win_pred.parameters()),
        lr=lr,weight_decay=wd
    )
    criterion = torch.nn.BCEWithLogitsLoss()

    # === Loaders ===


    train_loader_ngh = LastNeighborLoader(num_nodes=num_nodes, size=25, device=device)
    eval_loader_ngh  = LastNeighborLoader(num_nodes=num_nodes, size=25, device=device)

    assoc = torch.empty(num_nodes, dtype=torch.long, device=device)



    threshold = [0.6,0.65,0.7,0.75,0.8]
    num_epochs = 200

    import random

    train_variants = [
        (train_loader, full_data, train_data),

    ]
    patience = 10         # nombre d'epochs sans amélioration avant arrêt
    min_delta = 1e-4      # amélioration minimale pour reset la patience
    best_val_loss = 100
    epochs_no_improve = 0

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.9)
    hparams = {
    "learning_rate": lr,
    "weight_decay": wd,
    "memory_dim": memory_dim,
    "time_dim": time_dim,
    "embedding_dim": embedding_dim,
    "in_channels": in_channels,
    "hidden_channels": hidden_channels,
    "num_layers": num_layers,
    "heads": heads,
    "dropout": dropout,
    "hidden": hidden
    }

    run_dir = f"runs/run{run_id}_lr{lr}_wd{wd}"
    os.makedirs(run_dir.replace("runs/", "models/"), exist_ok=True)
    history = History(outdir=run_dir,hparams=hparams)

    train_losses, train_aps, train_prec = [], [], []
    val_losses,   val_aps,   val_prec  = [], [], []

    for epoch in range(1, num_epochs + 1):
        alpha = compute_alpha(epoch, num_epochs)

        loader, full, train_data_split = random.choice(train_variants)

        loss, ap, prec = train_debug2(
            loader, memory, gnn, win_pred, full, train_loader_ngh, eval_loader_ngh,
            optimizer, device, assoc, train_data_split, alpha
        )

        train_losses.append(loss)
        train_aps.append(ap)
        train_prec.append(prec)

        val_ap, val_loss, prec_v, prec_at, well_dates, bad_dates = evaluate(
            test_loader, memory, gnn, win_pred, full_data, eval_loader_ngh,
            assoc, device, threshold, alpha
        )

        val_losses.append(val_loss)
        val_aps.append(val_ap)
        val_prec.append(prec_v)

        if val_loss < best_val_loss - min_delta:
            best_val_loss = val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            
        if epochs_no_improve >= patience:
            print(f"⏹️ Early stopping (aucune amélioration après {patience} epochs).")
            break

        # --- LOG + SAUVEGARDE INCRÉMENTALE  ---
        history.log_epoch(
            epoch=epoch,
            train_loss=loss, train_ap=ap, train_prec=prec,
            val_loss=val_loss, val_ap=val_ap, val_prec=prec_v,
            prec_at=prec_at
        )
        
        history.save_tables()
        history.save_plots()
        torch.save({
        "memory_state": memory.state_dict(),
        "gnn_state": gnn.state_dict(),
        "win_pred_state": win_pred.state_dict(),
        "optimizer_state": optimizer.state_dict()
    }, f"models/run{run_id}_lr{lr}_wd{wd}/epoch{epoch}.pth")

    
    history.save_all()
    print("Courbes et CSV sauvegardés dans", history.outdir.resolve())


/home/romain/tensorflow_project/tennis/env/lib/python3.12/site-packages/torch/_compile.py:32: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  return disable_fn(*args, **kwargs)



=== RUN 1 | lr=0.001 | wd=0.0001 | hidden=[512, 64] ===
TGNMemory params: 464,192
MultiLayerTimeAwareGNN params: 174,528
SmallWinPredictor params: 190,849
Total parameters: 829,569


Evaluating: 100%|██████████| 93/93 [00:01<00:00, 85.13batch/s]


⏹️ Early stopping (aucune amélioration après 10 epochs).


/home/romain/tensorflow_project/tennis/env/lib/python3.12/site-packages/torch/_compile.py:32: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  return disable_fn(*args, **kwargs)


...

Evaluating: 100%|██████████| 93/93 [00:01<00:00, 83.89batch/s]


⏹️ Early stopping (aucune amélioration après 10 epochs).


/home/romain/tensorflow_project/tennis/env/lib/python3.12/site-packages/torch/_compile.py:32: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  return disable_fn(*args, **kwargs)


Courbes et CSV sauvegardés dans /home/romain/tensorflow_project/tennis/random-forest-tennis/random-forest-tennis/runs/run7_lr0.0004_wd0.0005

=== RUN 8 | lr=0.0004 | wd=0.0005 | hidden=[256, 64] ===
TGNMemory params: 464,192
MultiLayerTimeAwareGNN params: 174,528
SmallWinPredictor params: 108,161
Total parameters: 746,881


Training:  91%|█████████ | 2616/2889 [01:10<00:07, 35.33batch/s]